In [ ]:
import numpy as np
from scipy.integrate import tplquad
from plotly import __version__
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import plotly.graph_objs as go
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
init_notebook_mode(connected=True)

In [ ]:
k = 8.987551E9

In [ ]:
#outputs force element in direction i, given coulomb's constant k, test charge position p, and density rho for a sphere
def Force_element_sphere(r,theta,phi):
    r_vector = np.array([r*np.sin(phi)*np.cos(theta),r*np.sin(phi)*np.sin(theta),r*np.cos(phi)])
    distance= p-r_vector
    dV = np.linalg.norm(r_vector)**2*np.sin(phi)
    dq = dV*rho
    dF = dq*k*distance/(np.linalg.norm(distance)**3)
    return dF[i]

In [ ]:
def Force_element_disk(r,theta,phi):
    r_vector = np.array([r*np.sin(phi)*np.cos(theta),r*np.sin(phi)*np.sin(theta),r*np.cos(phi)])
    distance= p-r_vector
    dV = np.linalg.norm(r_vector)**2*np.sin(phi)
    dq = dV*rho
    dF = dq*k*distance/(np.linalg.norm(distance)**3)
    return dF[i]

In [ ]:
class surface: 
    def __init__(self, density,shape,linear_dim):
        self.rho = density
        self.s = shape
        self.dim= linear_dim

In [ ]:
class test_charge:
    def __init__(self,position):
        self.p = np.array(position)
    def force(self,surface):
        F = list()
        if surface.s == 'sphere':
            global p,rho,i
            rho = surface.rho
            p = self.p
            for j in range(3):
                i=j
                #tplquad returns the triple integral of the force
                F.append(tplquad(Force_element_sphere,0,np.pi,lambda phi: 0, lambda phi: 2*np.pi, lambda phi,theta: 0, lambda phi, theta: surface.dim)[0])

                u = np.linspace(0, 2 * np.pi, 100)
                v = np.linspace(0, np.pi, 100)
                x = surface.dim*np.outer(np.cos(u), np.sin(v))
                y = surface.dim*np.outer(np.sin(u), np.sin(v))
                z = surface.dim*np.outer(np.ones(np.size(u)), np.cos(v))

        
                trace = go.Surface(
                colorscale = [[0,'rgb(15,7,122)'],[1,'rgb(80,80,230)']],
                z = z,
                x= x,
                y = y)
        trace2 = go.Scatter3d(
                x = [p[0],p[0]+F[0]*(0.5*10**(-9))],
                y= [p[1],p[1]+F[1]*(0.5*10**(-9))],
                z=[p[2],p[2]+F[2]*(0.5*10**(-9))],
                marker = dict(
                        size=2,
                        color= 'rgb(21,117,22)',
                        symbol= 'diamond'
                        ),
                line=dict(
                        width=5,
                        color='rgb(21,117,22)',
                        )
                )
        
        trace3 = go.Scatter3d(
                x = [p[0]],
                y= [p[1]],
                z=[p[2]],
                marker = dict(
                        size=5,
                        color= 'rgb(214,11,8)'
                        )
                )
        
        data = [trace,trace2,trace3]
        layout = go.Layout(
                width = 800,height=800,
                autosize=True,
                scene = dict(
                xaxis = dict(range=[-5, 5], autorange=False, zeroline=False),
                yaxis= dict(range=[-5, 5], autorange=False, zeroline=False),
                zaxis=dict(range=[-5, 5], autorange=False, zeroline=False),
                aspectmode = 'cube'
                ),
                title = 'Test Charge'

                )
        figure = go.Figure(data=data,layout=layout)
        iplot(figure)

In [ ]:
def make_plot(density,shape,linear_dim,position):
    sphere1 = surface(density,shape,linear_dim)
    test = test_charge(position)
    test.force(sphere1)

In [ ]:
interact(make_plot,density=(0,2.,0.2),shape=fixed('sphere'),linear_dim=fixed(1.),position=fixed([2.,2.,2.,]))